In [107]:
import urllib2, time, random, re
from bs4 import BeautifulSoup
import pandas as pd

In [96]:
hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
           'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
           'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
           'Accept-Encoding': 'none',
           'Accept-Language': 'en-US,en;q=0.8','Connection': 'keep-alive'}

soup = BeautifulSoup(urllib2.urlopen(urllib2.Request("http://ehealthforum.com/health/doctor_questions_259.html", headers=hdr)).read())
url_list = []
for post in soup.find_all("div", class_="dq_topic_content_title"):
    if("Answered by" in post.find("span",class_="dq_topic_author").text):
        url_list.append(post.find("a")['href'])

The workflow will be to grab each forum post that has a 

In [ ]:
df = pd.DataFrame()
for url in url_list:
    try:
        soup = BeautifulSoup(urllib2.urlopen(urllib2.Request(url, headers=hdr)).read())
        for reply in soup.find_all("div", class_="vt_post_holder"):
            store = {"post":url}
            if(reply.find("div", class_="vt_first_message_body")):
                store["response_type"]="asker"
                store["response_time"]=reply.find("span",class_="vt_first_timestamp").text
                store["response"]=re.sub('\s+',' ',reply.find("div",class_="vt_post_body").text)
                store["user_name"]=reply.find("span",class_="vt_asked_by_user").text
                store["user_profile"]=reply.find("span",class_="vt_asked_by_user").find("a")["href"]
            if(reply.find("span",class_="postfix-md")):
                store["response_type"]="doctor"
                store["response_time"]=reply.find("div",class_="vt_reply_timestamp").text
                store["response"]=re.sub('\s+',' ',reply.find("div",class_="vt_post_body").text)
                store["user_name"]=reply.find("div",class_="vt_asked_by_user").text
                store["user_profile"]=reply.find("div",class_="vt_asked_by_user").find("a")["href"]
            df=df.append(store,ignore_index=True)
        time.sleep(random.randint(15,45))
    except urllib2.HTTPError, e:
        pass
df.to_csv("skin_data_1.csv")

u' Thank you for asking at Ehealth forum! I read your question and i understand your concern. Its likely a pinched nerve , common after sleep due to prolonged sleeping postures in one position, Just take some tylenol , exercise for some time, and it will be back to normal. I hope it helps. Stay in touch with your healthcare provider for further guidance as our answers are just for education and counselling purposes and cannot be an alternative to actual visit to a doctor. Take care Khan '